In [2]:
import pandas as pd
import json
from google.cloud import storage

In [18]:
BUCKET = "rjr-portal-da-transparencia"

In [19]:
client = storage.Client()
bucket = client.get_bucket(BUCKET)

In [20]:
def save_maps(name, maps):
    with open(f'{name}_maps.json', 'w', encoding='utf-8') as file:
        json.dump(maps, file, ensure_ascii=False)

    blob = bucket.blob(f"aux_tables/{name}_maps.json")
    blob = blob.upload_from_filename(f'{name}_maps.json')

In [21]:
df = pd.read_csv(f"gs://{BUCKET}/aux_tables/ufs.csv", 
                 engine="python", sep=',', quotechar='"', header=0, encoding="utf8")
maps = {}
names = ["CO_REGIAO", "CO_UF", "CO_MESORREGIAO", 
        "CO_MICRORREGIAO", "CO_MUNICIPIO"]
cols = [[0, 1], [2, 3], [2, 5, 6], [2, 7, 8], [9, 10]]
names_cols = zip(names, cols)


for name, cols_ in names_cols:
    
    df_ = df.iloc[:, cols_].drop_duplicates()
    if len(cols_) == 2:
        map_ = dict(df_.values.tolist())
    elif name == "CO_MESORREGIAO":
        map_ = {f"{col1}{col2:02}": col3
                       for col1, col2, col3 
                       in df_.values}
    else:
        map_ = {f"{col1}{col2:03}": col3
                       for col1, col2, col3 
                       in df_.values} 
    maps[name] = map_

In [22]:
maps["CO_UF_NASC"] = maps["CO_UF"] 
maps["CO_MUNICIPIO_NASC"] = maps["CO_MUNICIPIO"] 
maps["CO_UF_END"] = maps["CO_UF"] 
maps["CO_MUNICIPIO_END"] = maps["CO_MUNICIPIO"] 

In [23]:
pais_origem = pd.read_csv(f"gs://{BUCKET}/aux_tables/CO_PAIS_ORIGEM.csv", 
                 engine="python", sep=',', quotechar='"', header=0, encoding="utf8")

In [24]:
maps["CO_PAIS_ORIGEM"] = dict(pais_origem.values.tolist())
maps["CO_PAIS_RESIDENCIA"] = maps["CO_PAIS_ORIGEM"] 

In [25]:
save_maps("regioes", maps)